In [1]:
from mindquantum.algorithm.error_mitigation import zne
from mindquantum.simulator import Simulator
from mindquantum.core.circuit import Circuit
from mindquantum.core.operators import Hamiltonian, QubitOperator

In [138]:
from mindquantum.core.gates import RX, RY, RZ
import numpy as np

def rb_circ(n_gate):
    circ = Circuit()
    for _ in range(n_gate):
        circ += np.random.choice([RX,RY,RZ])(np.random.uniform(-1,1)).on(0)
    return circ

circ = rb_circ(50)
circ.svg()

In [3]:
ham = Hamiltonian(QubitOperator('Z0'))
ham

1 [Z0]

In [139]:
noiseless_sim = Simulator('mqmatrix', circ.n_qubits)
e0 = noiseless_sim.get_expectation(ham, circ)
e0

(0.4264802181758561+0j)

In [81]:
from mindquantum.core.circuit.channel_adder import ChannelAdderBase, BitFlipAdder
from mindquantum.device.chip import NaiveChip
from mindquantum.core.gates import BasicGate, DepolarizingChannel


class DepolarizingAdder(ChannelAdderBase):
    """Add Depolarizing channel after quantum gate."""

    def __init__(self, p: float = None, with_ctrl=True, device: NaiveChip = None, add_after: bool = True):
        """Initialize a DepolarizingAdder."""
        super().__init__(add_after=add_after)
        self.with_ctrl = True
        self.p = p
        self.device = device
        self.with_ctrl = with_ctrl

    def __repr__(self):
        """Return string expression of adder."""
        if self.device is None:
            return f"DepolarizingAdder<p={self.p}, with_ctrl={self.with_ctrl}>"
        return f"DepolarizingAdder<device={self.device}, with_ctrl={self.with_ctrl}>"

    def _handler(self, g: BasicGate):
        """Action you will do if a gate is acceptable."""
        circ = Circuit()
        for q in g.obj_qubits + (g.ctrl_qubits if self.with_ctrl else []):
            if self.device is not None:
                circ += self.device.gene_channel(self.device, g, DepolarizingChannel, self.with_ctrl,
                                                 DepolarizingChannel(self.p)).on(q)
            else:
                circ += DepolarizingChannel(self.p).on(q)
        return circ


In [144]:
from mindquantum.simulator.noise import NoiseBackend
import numpy as np

noise_sim = Simulator(NoiseBackend('mqmatrix', circ.n_qubits, BitFlipAdder(0.001)))
e_noise = noise_sim.get_expectation(ham, circ)
e_noise

(0.3960689062783595+0j)

In [145]:
zne(circ, ham, noise_sim, np.linspace(1, 3, 3), 1)

[(0.3960689062783595+0j), (0.36627674268215454+0j), (0.34894266249916917+0j)]


(0.4383191532877839+0j)